# 패키지 로드

In [1]:
#!pip install shapely
#!pip install geopy
import time
from selenium import webdriver
import json
from shapely.geometry import Point
from shapely.geometry import mapping
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import numpy as np
import pandas as pd
from geopy.distance import great_circle
import os
import folium
from haversine import haversine
import matplotlib.pyplot as plt
#import pydeck as pdk
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import seaborn as sns
import random
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install colour
from colour import Color

In [3]:
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# 데이터 로드

In [4]:
busan_drag_train = pd.read_csv('../data/Busan/busan_drag_train.csv', encoding='cp949')
busan_anch_train = pd.read_csv('../data/Busan/busan_anch_train.csv', encoding='cp949')
busan_answer = pd.read_csv('../data/Busan/busan_answer.csv', encoding='cp949')

In [19]:
borderpoint = pd.read_excel("../박에스더/borderpoint.xlsx")

# borderpoint['위도']= borderpoint['location'].apply(lambda x : str(x)[3:].replace('N','').replace('E ',''). replace(' ','').split(',')[0])

# borderpoint['경도']= borderpoint['location'].apply(lambda x : str(x)[3:].replace('N','').replace('E ',''). replace(' ','').split(',')[1])

# borderpoint['위도']= borderpoint['위도'].apply(lambda x : float(str(x).split('-')[0])+float(str(x).split('-')[1])/60+ float(str(x).split('-')[2])/3600)

# borderpoint['경도']= borderpoint['경도'].apply(lambda x : float(str(x).split('-')[0])+float(str(x).split('-')[1])/60+ float(str(x).split('-')[2])/3600)

# borderpoint = borderpoint.drop('location',axis=1)

In [ ]:
len()

In [20]:
borderpoint

,name,위도,경도
0,N2,35.076444,129.026528
1,N2,35.076444,129.031500
2,N2,35.053000,129.031222
3,N2,35.052944,129.017278
4,N2,35.061472,129.024917
5,N2,35.074139,129.024917
6,N3,35.067139,129.034417
7,N3,35.067139,129.053083
8,N3,35.048861,129.080056
9,N3,35.044306,129.037083


# 시각화

In [41]:
# 점 그리는 함수
def draw_point(df_drag,df_answer,boat_num,num):
    df_drag= df_drag[df_drag['선박번호']==boat_num].reset_index(drop=True)
    df_answer= df_answer[df_answer['선박번호']==boat_num].reset_index(drop=True)

    
    path = folium.Map(location=[35.1380, 129.1737], zoom_start=11)
    path.add_child(folium.LatLngPopup())
    
    # 정박지 그리기
    for i in borderpoint['name'].unique():
        data = borderpoint[borderpoint['name'] == i]
        data = data[['위도','경도']]
        polygon_coords = data
        folium.Polygon(locations=polygon_coords, color='blue', fill=True,
                fill_color='blue', fill_opacity=0.1).add_to(path)
    
    # 경로 색깔 지정
    colors = list(Color("black").range_to(Color("yellow"),num))
    for n, i in enumerate(colors):
        colors[n] = str(i)
        
    #첫번째 지점 그리기
    
    folium.CircleMarker([df_drag.loc[0,'위도'],df_drag.loc[0,'경도']],
                        popup=(df_drag.loc[0,'날짜_시간']),
                        radius=5,
                        color='white',
                        fill=True,
                        fill_opacity=0.7,
                        fill_color='white',
                        ).add_to(path)
    
    #정답 데이터 그리기
    lat = df_answer['위도']
    lon = df_answer['경도']
    time = df_answer['날짜_시간']
    
    folium.CircleMarker([lat,lon],
                           popup=(time),
                                radius=5,
                                color='white',
                                fill=True,
                                fill_opacity=0.7,
                                fill_color='white',
                               ).add_to(path)
    
    
    
    #경로 그리기
    for k in range(num):
        for i in range(int(k*len(df_drag)/num),int(k*len(df_drag)/num+len(df_drag)/num)):
            lat = df_drag['위도'][i]
            lon = df_drag['경도'][i]
            time = df_drag['날짜_시간'][i]
        
            folium.CircleMarker([lat,lon],
                           popup=(time),
                                radius=0.04,
                                color=colors[k],
                                fill=True,
                                fill_opacity=0.7,
                                fill_color=colors[k],
                               ).add_to(path)
        
    
    return path

In [169]:
draw_point(busan_drag_train,busan_answer,5,10)

# busan_drag_train

## 점사이 거리

In [142]:
def distance (df_drag, boat_num):
    df = df_drag[df_drag['선박번호'] == boat_num].reset_index(drop=True)

    
    for i in range(len(df)-1):
        df.loc[i+1,'점사이_거리']=haversine(
            (df.loc[i,'위도'],df.loc[i,'경도']),
            (df.loc[i+1,'위도'],df.loc[i+1,'경도']),
            unit = 'km')
    
    return df

In [167]:
result = pd.DataFrame()

for n in busan_drag_train['선박번호'].unique():
    df = distance(busan_drag_train, n)
    df['시간_차이'] = pd.to_datetime(df['날짜_시간']).diff()
    
    result = result.append(df)
    

# 결측치 처리

In [172]:
import numpy as np

In [188]:
result.loc[result['실침로']==-99.9,'실침로'] =np.NaN
result.loc[result['대지속력']==102.3,'대지속력'] =np.NaN
result.loc[result['선수미선']== -99,'선수미선'] =np.NaN

In [190]:
result

,선박번호,날짜_시간,날짜,시간,위도,경도,대지속력,실침로,선수미선,점사이_거리,시간_차이
0,1,2021-01-14 22:47:00,2021-01-14,22:47:00,35.049747,129.07212,0.5,123.8,NaN,NaN,NaT
1,1,2021-01-14 22:50:00,2021-01-14,22:50:00,35.049485,129.07243,0.2,131.7,NaN,0.040560,0 days 00:03:00
2,1,2021-01-14 22:56:00,2021-01-14,22:56:00,35.049452,129.07244,0.2,255.9,NaN,0.003781,0 days 00:06:00
3,1,2021-01-14 23:02:00,2021-01-14,23:02:00,35.049402,129.07225,0.1,280.3,NaN,0.018167,0 days 00:06:00
4,1,2021-01-14 23:11:00,2021-01-14,23:11:00,35.049100,129.07221,0.1,157.6,NaN,0.033778,0 days 00:09:00
...,...,...,...,...,...,...,...,...,...,...,...
756,87,2022-05-07 20:25:00,2022-05-07,20:25:00,35.053083,129.06133,1.3,26.0,97.0,0.004096,0 days 00:00:00
757,87,2022-05-07 20:25:00,2022-05-07,20:25:00,35.053083,129.06133,1.3,29.0,99.0,0.000000,0 days 00:00:00
758,87,2022-05-07 20:26:00,2022-05-07,20:26:00,35.053117,129.06140,1.2,48.0,106.0,0.007409,0 days 00:01:00
759,87,2022-05-07 20:26:00,2022-05-07,20:26:00,35.053117,129.06141,1.2,52.0,108.0,0.000910,0 days 00:00:00


# 합치기

In [1]:
import pandas as pd

In [49]:
drag = pd.read_csv('./busan_drag_train_total.csv', encoding='cp949')

In [50]:
drag 

,선박번호,날짜_시간,날짜,시간,위도,경도,대지속력,실침로,선수미선,점사이_거리,시간_차이,정박지,평균선회반경(m)
0,1,2021-01-14 22:47:00,2021-01-14,22:47:00,35.049747,129.07212,0.5,123.8,NaN,NaN,NaN,N3,322.0
1,1,2021-01-14 22:50:00,2021-01-14,22:50:00,35.049485,129.07243,0.2,131.7,NaN,0.040560,0 days 00:03:00,N3,322.0
2,1,2021-01-14 22:56:00,2021-01-14,22:56:00,35.049452,129.07244,0.2,255.9,NaN,0.003781,0 days 00:06:00,N3,322.0
3,1,2021-01-14 23:02:00,2021-01-14,23:02:00,35.049402,129.07225,0.1,280.3,NaN,0.018167,0 days 00:06:00,N3,322.0
4,1,2021-01-14 23:11:00,2021-01-14,23:11:00,35.049100,129.07221,0.1,157.6,NaN,0.033778,0 days 00:09:00,N3,322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219935,87,2022-05-07 20:25:00,2022-05-07,20:25:00,35.053083,129.06133,1.3,26.0,97.0,0.004096,0 days 00:00:00,N3,322.0
219936,87,2022-05-07 20:25:00,2022-05-07,20:25:00,35.053083,129.06133,1.3,29.0,99.0,0.000000,0 days 00:00:00,N3,322.0
219937,87,2022-05-07 20:26:00,2022-05-07,20:26:00,35.053117,129.06140,1.2,48.0,106.0,0.007409,0 days 00:01:00,N3,322.0
219938,87,2022-05-07 20:26:00,2022-05-07,20:26:00,35.053117,129.06141,1.2,52.0,108.0,0.000910,0 days 00:00:00,N3,322.0
